- Using the counts produced by count freqs.py, write a function that computes emission parameters

In [1]:
emission_par = []
# Count(y) = 345128 for 0
# Count(y) = 41072 for I-GENE

with open('gene.counts', encoding="utf-16") as file:
    for line in file:
        parts = line.strip().split(" ")
        if parts[1] == "WORDTAG" and parts[2] == "O":
            emission_par.append(float(parts[0]) / 345128 )
        elif parts[1] == "WORDTAG" and parts[2] == "I-GENE":
            emission_par.append(float(parts[0]) / 41072 )
        
emission_par[:10] # Display only the first 10

[0.00011589902876613894,
 0.009755800746389745,
 0.0006817296455005844,
 0.00021912738605375925,
 0.0285285459307851,
 0.0026782236073237243,
 0.10700720685625244,
 2.4347487339306584e-05,
 8.69242715746042e-06,
 4.6359611506455574e-05]

In [4]:
temp = [] # To store pairs that has count less than 5

with open('gene.counts', 'r', encoding="utf-16") as file:
    data = file.readlines()

for i in data:
    parts = i.split()
    if int(parts[0]) < 5:
        word = parts[-1]
        ne_type = parts[-2]
        temp.append((ne_type,word))

temp [:10] # Display only the first 10 rows

[('I-GENE', 'nucleotidase'),
 ('O', 'Pharmacologic'),
 ('O', '-]'),
 ('O', 'displaced'),
 ('O', 'Flurazepam'),
 ('O', 'hypnotic'),
 ('O', 'optimum'),
 ('O', 'Beta'),
 ('O', 'extracorporeal'),
 ('O', 'approximated')]

In [5]:
library = [] # To store all the data

with open('gene.train', 'r') as file:
    data = file.readlines()

print(data[:10])

['Comparison O\n', 'with O\n', 'alkaline I-GENE\n', 'phosphatases I-GENE\n', 'and O\n', '5 I-GENE\n', '- I-GENE\n', 'nucleotidase I-GENE\n', '\n', 'Pharmacologic O\n']


In [6]:
for i in data:
    if i.strip():
        parts = i.split()
        if (parts[1], parts[0]) in temp:
            parts[1] = '_RARE_'
        library.append(" ".join(parts) + "\n")
    else:
        library.append("\n")

print(len(library))
print(library[:10])

399995
['Comparison O\n', 'with O\n', 'alkaline I-GENE\n', 'phosphatases I-GENE\n', 'and O\n', '5 I-GENE\n', '- I-GENE\n', 'nucleotidase _RARE_\n', '\n', 'Pharmacologic _RARE_\n']


In [8]:
# Write file back to the file 
my_file = open("gene.train", "w")
my_file.writelines(library)
my_file = open("gene.train")
content = my_file.read()
my_file.close()

In [12]:
from collections import defaultdict
emission_par = []

emi_prob = defaultdict(int)
# Count(y) = 315445 for 0
# Count(y) = 30503 for I-GENE
# Count(y) = 40252 for _RARE_

with open('gene.counts2', encoding="utf-16") as file:
    for line in file:
        parts = line.strip().split(" ")
    
        word = parts[-1]
        ne_type = parts[2]

        if parts[1] == "WORDTAG" and ne_type == "O":
            if word in emi_prob:
                temp = float(parts[0]) / 315445 
                if emi_prob.get(word)[1] < temp:
                    emi_prob[word] = [ne_type,temp]
            else:
                emi_prob[word] = [ne_type,temp]

        elif parts[1] == "WORDTAG" and ne_type == "I-GENE":
            if word in emi_prob:
                temp = float(parts[0]) / 30503 
                if emi_prob.get(word)[1] < temp:
                    emi_prob[word] = [ne_type,temp]
            else:
                emi_prob[word] = [ne_type,temp]

        elif parts[1] == "WORDTAG" and ne_type == "_RARE_":
            if word in emi_prob:
                temp = float(parts[0]) / 40252 
                if emi_prob.get(word)[1] < temp:
                    emi_prob[word] = [ne_type,temp]
            else:
                emi_prob[word] = [ne_type,temp]

emi_prob

defaultdict(int,
            {'Comparison': ['O', 2.4843486037960847e-05],
             'with': ['O', 2.4843486037960847e-05],
             'alkaline': ['I-GENE', 2.4843486037960847e-05],
             'phosphatases': ['_RARE_', 4.9686972075921695e-05],
             'and': ['I-GENE', 0.0043930105235550606],
             '5': ['O', 0.002761178652379971],
             '-': ['O', 0.027754442137298104],
             'nucleotidase': ['_RARE_', 2.4843486037960847e-05],
             'Pharmacologic': ['_RARE_', 2.4843486037960847e-05],
             'aspects': ['O', 2.4843486037960847e-05],
             'of': ['I-GENE', 0.0031472314198603417],
             'neonatal': ['O', 2.4843486037960847e-05],
             'hyperbilirubinemia': ['O', 2.4843486037960847e-05],
             '.': ['I-GENE', 0.005409304002884962],
             'When': ['O', 2.4843486037960847e-05],
             'CSF': ['I-GENE', 0.0018031013342949874],
             '[': ['_RARE_', 9.937394415184339e-05],
             'HCO3': ['O

In [18]:
stor = [] # To store all the data

with open('gene.dev', 'r') as file:
    data = file.readlines()

for i in data:
    if i.strip():
        parts = i.split()
        word = parts[0]
        if (word) in emi_prob:
            parts.append(emi_prob[word][0])
        else:
            parts.append('_RARE_')    
        stor.append(" ".join(parts) + "\n")
    else:
        stor.append("\n") 

stor[:20]

['BACKGROUND O\n',
 ': I-GENE\n',
 'Ischemic O\n',
 'heart O\n',
 'disease O\n',
 'is O\n',
 'the I-GENE\n',
 'primary O\n',
 'cause O\n',
 'of I-GENE\n',
 'morbidity O\n',
 'and I-GENE\n',
 'mortality O\n',
 'among O\n',
 'diabetics _RARE_\n',
 ', O\n',
 'especially O\n',
 'those O\n',
 'who O\n',
 'became O\n']

In [19]:
# Write file back to the file 
f = open("gene_dev.p1.out", "x")
f = open("gene_dev.p1.out", "w")
f.writelines(stor)
f.close()